In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lifetimes
from zipfile import ZipFile

#Let's make this notebook reproducible 
np.random.seed(42)

import random
random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# open zipped dataset and save to df's
with ZipFile("data/customer_data_base.zip") as z:
   # open the orders csv file in the dataset
   with z.open("olist_orders_dataset.csv") as a:    
      # read the dataset as dataframe "df"
      orders_df = pd.read_csv(a)
        
   # open the customers csv file in the dataset
   with z.open("olist_customers_dataset.csv") as b:    
      # read the dataset as dataframe "df"
      customers_df = pd.read_csv(b)     
        
   # open the customers csv file in the dataset
   with z.open("olist_order_payments_dataset.csv") as c:    
      # read the dataset as dataframe "df"
      payments_df = pd.read_csv(c)     


In [3]:
print("orders_df","customers_df","payments_df")

orders_df customers_df payments_df


In [4]:
#creating an orders df with unique customer id
cols = ['customer_id', 'order_id', 'order_purchase_timestamp']
orders = orders_df[cols]
orders = orders.set_index('customer_id')
orders.drop_duplicates(inplace=True)

In [5]:
#creating a customers df with unique customer id
cols = ['customer_id', 'customer_unique_id']
customers = customers_df[cols]
customers = customers.set_index('customer_id')

In [6]:
#creating a payments df with unqique order ids
cols = ['order_id', 'payment_value']
payment = payments_df[cols]
payment = payment.set_index('order_id')
payment.drop_duplicates(inplace=True)

In [7]:
#I don't know
elog = pd.concat([orders,customers], axis=1, join='inner')
elog.reset_index(inplace=True)

cols = ['customer_unique_id', 'order_purchase_timestamp']
elog = elog[cols]

elog['order_purchase_timestamp'] = pd.to_datetime(elog['order_purchase_timestamp'])
elog['order_date'] = elog.order_purchase_timestamp.dt.date
elog['order_date'] = pd.to_datetime(elog['order_date'])

cols = ['customer_unique_id', 'order_date']
elog = elog[cols]

elog.columns = ['CUSTOMER_ID', 'ORDER_DATE']


elog.info()
display(elog.sample(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CUSTOMER_ID  99441 non-null  object        
 1   ORDER_DATE   99441 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 1.5+ MB


,CUSTOMER_ID,ORDER_DATE
52263,2d1bf256227e4d22d10ea6c0b81809d7,2018-06-12
46645,12bf514b8d413d8cbe66a2665f4b724c,2018-01-20
37546,83c6df0d47130de38c99cebe96521e8a,2018-06-16
94756,29b186723b197669f69b7d63c3e27c07,2017-08-30
14771,a59129ed35da4c3e3f2a005b4c6582fc,2017-08-10


### From lifetimes documentation

In [8]:
#import sample data
from lifetimes.datasets import load_cdnow_summary
data = load_cdnow_summary(index_col=[0])

print(data.head())

    frequency  recency      T
ID                           
1           2    30.43  38.86
2           1     1.71  38.86
3           0     0.00  38.86
4           0     0.00  38.86
5           0     0.00  38.86


In [9]:
#df 
data

,frequency,recency,T
ID,,,
1,2,30.43,38.86
2,1,1.71,38.86
3,0,0.00,38.86
4,0,0.00,38.86
5,0,0.00,38.86
...,...,...,...
2353,0,0.00,27.00
2354,5,24.29,27.00
2355,0,0.00,27.00


In [10]:
# similar API to scikit-learn and lifelines
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(data['frequency'], data['recency'], data['T'])
print(bgf)

<lifetimes.BetaGeoFitter: fitted with 2357 subjects, a: 0.79, alpha: 4.41, b: 2.43, r: 0.24>


In [13]:
bgf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
r,0.242593,0.012557,0.217981,0.267205
alpha,4.413532,0.378221,3.672218,5.154846
a,0.792886,0.185719,0.428877,1.156895
b,2.425752,0.705345,1.043276,3.808229
